In [ ]:
IBM Capstone - ITUB4 TS Forecasting using News

In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='********-****-******-****-**********', project_access_token='p-************************************')
pc = project.project_context

In [ ]:
!pip install yfinance==0.1.54
!pip install pandas-datareader==0.8.1

In [1]:
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import pandas as pd
import datetime 
today = date.today()

/home/anthony/Capstone Advanced DS/github/Deployment/venv/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
date_time = '2005-04-05'
start = datetime.datetime.strptime(date_time, '%Y-%d-%m') 

In [3]:
### Get the datas (Itau share price, ibovespa, USDBRL)
TS_itau = pdr.get_data_yahoo('itub4.sa', start=start, end=today).reset_index()
BVSP = pdr.get_data_yahoo('^BVSP', start=start, end=today).reset_index()
USDBRL = pdr.get_data_yahoo('USDBRL=X', start=start, end=today).reset_index()

### Cleanning and group by
TS_itau= TS_itau[['Date','Open','Close']]
BVSP= BVSP[['Date','Open']]
USDBRL= USDBRL[['Date','Open']]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
TS_itau.columns = ['Date','Itau_open','Itau_Close']
BVSP.columns = ['Date','BVSP_open']
USDBRL.columns = ['Date','USDBRL_open']
result = pd.merge(TS_itau, BVSP, on='Date')
result = pd.merge(result, USDBRL, on='Date')

In [5]:
s = result['Itau_Close']
lag_col = pd.concat([s.shift(), s.shift(2),s.shift(3)], axis=1)
lag_col.columns = ['lag_1','lag_2','lag_3']
result = pd.concat([result, lag_col], axis=1, sort=False).dropna()

In [6]:
result.head()

,Date,Itau_open,Itau_Close,BVSP_open,USDBRL_open,lag_1,lag_2,lag_3
3,2006-03-29,10.9282,10.8934,36684.0,2.2270,10.9282,11.1600,11.4829
4,2006-03-30,11.0093,10.8405,37493.0,2.2117,10.8934,10.9282,11.1600
5,2006-03-31,10.8620,10.5971,37783.0,2.1853,10.8405,10.8934,10.9282
6,2006-04-03,10.7047,11.0358,37952.0,2.1618,10.5971,10.8405,10.8934
7,2006-04-04,11.0789,11.2263,38718.0,2.1370,11.0358,10.5971,10.8405


In [ ]:
# overwrite the TS data
project.save_data(data=result.to_csv(), file_name='ts_data',overwrite=True)